In [1]:
import sklearn
import pandas as pd
import numpy as np

In [8]:
from pathlib import Path
import pandas as pd
import tarfile
import urllib.request

def load_housing_data():
    tarball_path = Path("datasets/titanic.tgz")
    if not tarball_path.is_file():
        Path("datasets").mkdir(parents=True, exist_ok=True)
        url = "https://homl.info/titanic.tgz"
        urllib.request.urlretrieve(url, tarball_path)
        with tarfile.open(tarball_path) as titanic:
            titanic.extractall(path="datasets")
    return pd.read_csv(Path("datasets/titanic/train.csv")), pd.read_csv(Path("datasets/titanic/test.csv"))

train, test = load_housing_data()

In [9]:
X_train = train.copy()
X_train = X_train.drop('Survived', axis=1)
X_train.set_index('PassengerId', inplace=True)
X_test = test.copy()
X_test.set_index('PassengerId', inplace=True)

In [5]:
# fill average ages for nan value
X_train['Age'] = X_train['Age'].fillna(X_train['Age'].mean())
X_train.drop('Cabin', axis=1, inplace=True)
X_train.dropna(subset='Embarked', axis=0, inplace=True)

In [13]:
X_train['Age']

PassengerId
1      22.0
2      38.0
3      26.0
4      35.0
5      35.0
       ... 
887    27.0
888    19.0
889     NaN
890    26.0
891    32.0
Name: Age, Length: 891, dtype: float64

In [15]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="median")
X_train_num = X_train.select_dtypes(include=[np.number])
imputer.fit(X_train_num)


SimpleImputer(strategy='median')

In [16]:
imputer.statistics_

array([ 3.    , 28.    ,  0.    ,  0.    , 14.4542])

In [17]:
imputer.feature_names_in_

array(['Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype=object)

In [19]:
imputer.get_feature_names_out()

array(['Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype=object)